In [1]:
import logging
import pandas as pd
import numpy as np
import mysql.connector
import datetime
import os

# Set up basic logging
logging.basicConfig(
    filename='etl_log.txt',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

In [2]:
csv_file_path = 'employees1.csv'  # Make sure this file is in your working directory

df = pd.read_csv(csv_file_path)

print("Raw data loaded:")
print(df.head())
print("Column names:", df.columns.tolist())

logging.info("CSV loaded successfully.")

Raw data loaded:
   EMPLOYEE_ID FIRST_NAME  LAST_NAME     EMAIL  PHONE_NUMBER  HIRE_DATE  \
0          198     Donald   OConnell  DOCONNEL  650.507.9833  21-Jun-07   
1          199    Douglas      Grant    DGRANT  650.507.9844  13-Jan-08   
2          200   Jennifer     Whalen   JWHALEN  515.123.4444  17-Sep-03   
3          201    Michael  Hartstein  MHARTSTE  515.123.5555  17-Feb-04   
4          202        Pat        Fay      PFAY  603.123.6666  17-Aug-05   

     JOB_ID  SALARY  
0  SH_CLERK    2600  
1  SH_CLERK    2600  
2   AD_ASST    4400  
3    MK_MAN   13000  
4    MK_REP    6000  
Column names: ['EMPLOYEE_ID', 'FIRST_NAME', 'LAST_NAME', 'EMAIL', 'PHONE_NUMBER', 'HIRE_DATE', 'JOB_ID', 'SALARY']


In [3]:
# Fill missing values
df.fillna({
    'EMAIL': 'not_provided@example.com',
    'PHONE_NUMBER': '0000000000',
    'HIRE_DATE': '01-Jan-00',
    'SALARY': 0
}, inplace=True)

# Standardize column names
df.columns = [col.strip().lower().replace(' ', '_') for col in df.columns]
print("Cleaned column names:", df.columns.tolist())

Cleaned column names: ['employee_id', 'first_name', 'last_name', 'email', 'phone_number', 'hire_date', 'job_id', 'salary']


In [4]:
# Convert 'hire_date' to datetime
df['hire_date'] = pd.to_datetime(df['hire_date'], format='%d-%b-%y', errors='coerce')

# Replace invalid dates with default
df['hire_date'] = df['hire_date'].fillna(pd.to_datetime('2000-01-01'))

# Convert 'salary' to numeric
df['salary'] = pd.to_numeric(df['salary'], errors='coerce').fillna(0).astype(int)

logging.info("Data cleaning completed.")

In [5]:
# Connect to MySQL
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Madhu@2311",
    database="employee"  # Replace with your actual DB name
)

cursor = mydb.cursor()

# Create table if it doesn't exist
cursor.execute("""
    CREATE TABLE IF NOT EXISTS salary_2 (
        empid INT PRIMARY KEY,
        firstname VARCHAR(50),
        lastname VARCHAR(50),
        email VARCHAR(100),
        phone VARCHAR(20),
        hire_date DATE,
        job_id VARCHAR(20),
        salary INT
    )
""")

In [6]:
for index, row in df.iterrows():
    sql = """
        INSERT INTO salary_2 (
            empid, firstname, lastname, email, phone, hire_date, job_id, salary
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE
            firstname=VALUES(firstname),
            lastname=VALUES(lastname),
            email=VALUES(email),
            phone=VALUES(phone),
            hire_date=VALUES(hire_date),
            job_id=VALUES(job_id),
            salary=VALUES(salary)
    """

    values = (
        int(row['employee_id']),
        row['first_name'],
        row['last_name'],
        row['email'],
        row['phone_number'],
        row['hire_date'].date(),
        row['job_id'],
        int(row['salary'])
    )

    cursor.execute(sql, values)

mydb.commit()
cursor.close()
mydb.close()

logging.info("ETL process completed successfully.")
print("ETL process completed successfully.")

ETL process completed successfully.


In [7]:
print("Current Working Directory:", os.getcwd())

Current Working Directory: E:\


In [1]:
!jupyter nbconvert --to script ETL.ipynb

[NbConvertApp] Converting notebook ETL.ipynb to script
[NbConvertApp] Writing 2934 bytes to ETL.py
